In [15]:
pip install pandas sqlalchemy


Note: you may need to restart the kernel to use updated packages.


In [16]:
import pandas as pd
from sqlalchemy import create_engine


In [17]:
orders = pd.read_csv("orders.csv")
print(orders.head())


   order_id  user_id  restaurant_id  order_date  total_amount  \
0         1     2508            450  18-02-2023        842.97   
1         2     2693            309  18-01-2023        546.68   
2         3     2084            107  15-07-2023        163.93   
3         4      319            224  04-10-2023       1155.97   
4         5     1064            293  25-12-2023       1321.91   

                  restaurant_name  
0               New Foods Chinese  
1  Ruchi Curry House Multicuisine  
2           Spice Kitchen Punjabi  
3          Darbar Kitchen Non-Veg  
4       Royal Eatery South Indian  


In [18]:
users = pd.read_json("users.json")
print(users.head())


   user_id    name       city membership
0        1  User_1    Chennai    Regular
1        2  User_2       Pune       Gold
2        3  User_3  Bangalore       Gold
3        4  User_4  Bangalore    Regular
4        5  User_5       Pune       Gold


In [19]:
engine = create_engine("sqlite:///restaurants.db")

restaurants = pd.read_sql("SELECT * FROM restaurants", engine)
print(restaurants.head())


   restaurant_id restaurant_name  cuisine  rating
0              1    Restaurant_1  Chinese     4.8
1              2    Restaurant_2   Indian     4.1
2              3    Restaurant_3  Mexican     4.3
3              4    Restaurant_4  Chinese     4.1
4              5    Restaurant_5  Chinese     4.8


In [21]:
merged_1 = pd.merge(
    orders,
    users,
    on="user_id",
    how="left"
)


In [22]:
final_df = pd.merge(
    merged_1,
    restaurants,
    on="restaurant_id",
    how="left"
)


In [23]:
final_df.to_csv("final_food_delivery_dataset.csv", index=False)


In [ ]:
print(final_df.columns)
print(final_df.head())


In [24]:
import pandas as pd

df = pd.read_csv("final_food_delivery_dataset.csv")

result = (
    df.groupby("restaurant_name_x")
      .agg(
          avg_order_value=("total_amount", "mean"),
          order_count=("order_id", "count")
      )
)

filtered = result[result["order_count"] < 20]
filtered.sort_values("avg_order_value", ascending=False).head()


,avg_order_value,order_count
restaurant_name_x,,
Hotel Dhaba Multicuisine,1040.222308,13
Sri Mess Punjabi,1029.180833,12
Ruchi Biryani Punjabi,1002.140625,16
Sri Delights Pure Veg,989.467222,18
Classic Kitchen Family Restaurant,973.167895,19


In [25]:
df.groupby(["membership", "cuisine"])["total_amount"] \
  .sum() \
  .sort_values(ascending=False)


membership  cuisine
Regular     Mexican    1072943.30
            Italian    1018424.75
Gold        Mexican    1012559.79
            Italian    1005779.05
Regular     Indian      992100.27
Gold        Indian      979312.31
            Chinese     977713.74
Regular     Chinese     952790.91
Name: total_amount, dtype: float64

In [26]:
df["order_date"] = pd.to_datetime(df["order_date"], dayfirst=True)

df["quarter"] = df["order_date"].dt.to_period("Q")

df.groupby("quarter")["total_amount"] \
  .sum() \
  .sort_values(ascending=False)


quarter
2023Q3    2037385.10
2023Q4    2018263.66
2023Q1    1993425.14
2023Q2    1945348.72
2024Q1      17201.50
Freq: Q-DEC, Name: total_amount, dtype: float64